## Ανάλυση Κανόνων Συσχετίσεων
* Δηλώνω τις βιβλιοθήκες που θα χρησιμοποιήσω

In [1]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules

* fact_table.txt -> το αρχείο έχει εξαχθεί από το data warehouse και αντιστοιχεί στον πίνακα fact Table όπως προέκυψε μετά τη διαδικασία ETL

In [2]:
data = pd.read_csv('fact_table.txt', sep=",")

* Διώχνω όποιο βιβλίο έχει αγοραστεί λιγότερο από 100 φορές

In [3]:
data_new = data.groupby('Book').filter(lambda x : len(x)>100)

* Κρατάω μόνο όσους χρήστες έχουν αγοράσει πάνω από τρεις φορές

In [4]:
final = data_new.groupby('Reader').filter(lambda x : len(x)>3).copy()

* Διατηρώ τους εναπομείναντες μοναδικούς τίτλους βιβλίων

In [5]:
titles = pd.DataFrame (final['Book'].unique().copy(), columns = ['Book_Title'])

* Διατηρώ τους εναπομείναντες μοναδικούς χρήστες

In [6]:
users = final['Reader'].unique().copy()

* Φτιάχνω ένα dictionary για τις ανάγκες του πίνακα. Δηλαδή ξεκινώντας από το 1 εως τον αριθμό του κάθε χρήστη και αντιστοιχώ user_id κατά αύξοντα αριθμό

In [7]:
users_dict = {}
index = 0
for i in users:
    users_dict[i] = index
    index = index + 1

* Φτιάχνω ένα data frame με γραμμές όσους και χρήστες και στήλες όσα και τα βιβλία. Γεμίζω το data frame με τιμές False

In [8]:
association_df = pd.DataFrame(False, index=np.arange(3393), columns=titles.Book_Title)

* Από το πίνακα final όπου περιέχει τα τελικά δεδομένα του Fact Table έτσι όπως εξήχθησαν από το Warehouse, ελέγχω τα βιβλία που διάβασε ο κάθε χρήστης. Με άλλα λόγια ελέγχω τον Fact Table και μόλις βρω ότι ο χρήστης έκανε κάποια κριτική για κάποιο βιβλίο πηγαίνω και σημειώνω στο association_df ότι ο συγκεκριμένος χρήστης όπου βρίσκεται στη γραμμή users_dict[row['Reader'] έχει διαβάσει το βιβλίο που βρίσκεται στη στήλη row['Book_Title']. Με αυτό το τρόπο έχω στο association_df την πληροφορία ποιος χρήστης διάβασε ποια βιβλία.

In [9]:
for index, row in final.iterrows():
  association_df.at[users_dict[row['Reader']], row['Book']] = True

* Από προεπιλογή, το apriori επιστρέφει τους δείκτες των στηλών των στοιχείων, οι οποίοι μπορεί να είναι χρήσιμοι σε επόμενες λειτουργίες, όπως η εξόρυξη κανόνων συσχέτισης. Για καλύτερη αναγνωσιμότητα, μπορούμε να ορίσουμε use_colnames=True για να μετατρέψουμε αυτές τις ακέραιες τιμές στα αντίστοιχα ονόματα στοιχείων

In [10]:
df = apriori(association_df, min_support=0.01, use_colnames=True, verbose=1, max_len= None, low_memory=False )
df

Processing 146306 combinations | Sampling itemset size 2

* Επιλέγω τους κορυφαίους 10 συνδυασμούς

In [ ]:
df_ar = association_rules(df, metric = "confidence")
final_results = df_ar.sort_values(by=['confidence'], ascending=False).iloc[:10][['antecedents', 'consequents', 'support', 'confidence', 'lift']]

* Τα τελικά αποτελέσματα με το book_id, απομένει ένα τελευταίο βήμα να εμφανίσω τα book_titles

In [ ]:
final_results

,antecedents,consequents,support,confidence,lift
328,"(36578, 36067, 130572, 36422)",(36120),0.010315,1.000000,24.586957
143,"(116954, 117171, 116979)",(116977),0.010315,1.000000,21.339623
125,"(82995, 123557, 82974)",(82757),0.017683,0.983607,26.073258
121,"(83596, 123557, 82974)",(82757),0.013852,0.979167,25.955566
118,"(82995, 83596, 123557)",(82757),0.012084,0.976190,25.876674
201,"(36578, 130572, 36422)",(36120),0.011494,0.975000,23.972283
295,"(82995, 83596, 123557, 82974)",(82757),0.011494,0.975000,25.845117
322,"(36578, 130572, 36277, 36422)",(36120),0.010610,0.972973,23.922444
232,"(36120, 36721, 36578)",(36277),0.010315,0.972222,24.078467
172,"(116954, 117171, 95630)",(116977),0.010315,0.972222,20.746855


* Για να εμφανίσω τους τίτλους βιβλίων χρειάζομαι το πίνακα dimension book_title_dim

In [ ]:
book_dim = pd.read_csv('bookdimension.csv', sep="|")

* Μετανομάζω και αφαιρώ κάποιες αχρείαστες στήλες

In [ ]:
book_dim.columns = book_dim.columns.str.replace('book_id', 'Book_Title')
book_dim.columns = book_dim.columns.str.replace('book_title', 'book_title_label')
book_dim = book_dim.drop(['ISBN', 'year_of_publication'], axis=1)

* Κάνω merge με τα βιβλία που χρησιμοποιήθηακν στον αλγόριθμο

In [ ]:
book_results = pd.merge(book_dim, titles, on=['Book_Title'])

* Και εμφανίζω τα τελικά αποτελέσματα

In [ ]:
for index, row in final_results.iterrows():
  asc =[]
  for r in row['antecedents']:
    for ind, rr in book_results.iterrows():
      if (r == rr['Book_Title']):
        asc.append(rr['book_title_label'])
        continue
  con =[]
  for r in row['consequents']:
    for ind, rr in book_results.iterrows():
      if (r == rr['Book_Title']):
        con.append(rr['book_title_label'])
        continue
  print(asc, con)
  print("----------->  " + 'support ' + str(row['support']) + " & confidence "+ str(row['confidence'])+ ' ')

['Seven Up (A Stephanie Plum Novel)', 'Three To Get Deadly : A Stephanie Plum Novel (A Stephanie Plum Novel)', 'Two for the Dough', 'Hot Six : A Stephanie Plum Novel (A Stephanie Plum Novel)'] ['Four To Score (A Stephanie Plum Novel)']
----------->  support 0.01031535514294135 & confidence 1.0 
['A Is for Alibi (Kinsey Millhone Mysteries (Paperback))', 'F Is for Fugitive (Kinsey Millhone Mysteries (Paperback))', 'C Is for Corpse (Kinsey Millhone Mysteries (Paperback))'] ['B Is for Burglar (Kinsey Millhone Mysteries (Paperback))']
----------->  support 0.01031535514294135 & confidence 1.0 
['Harry Potter and the Goblet of Fire (Book 4)', "Harry Potter and the Sorcerer's Stone (Book 1)", 'Harry Potter and the Prisoner of Azkaban (Book 3)'] ['Harry Potter and the Chamber of Secrets (Book 2)']
----------->  support 0.01768346595932803 & confidence 0.9836065573770492 
['Harry Potter and the Order of the Phoenix (Book 5)', "Harry Potter and the Sorcerer's Stone (Book 1)", 'Harry Potter and t